# Go faster: numba and multiprocessing

*Davide Gerosa (Milano-Bicocca)*

**Sources**: The data frog: https://thedatafrog.com/en/articles/make-python-fast-numba 

**Sources**: my own research

The usual criticism to python is that it's slow. The underlying reason is that python is an interpreted language and not a compiled language (but that's also why python it's nice and simple!).

Numpy largely solves the speed problem in most cases, but not everything can be written in a numpy-compatible form (or even if it can, you might need to rewrite substantial chunks of code). 

Today we look into two complementary strategies to speed up python.

# Part 1. Just-in-time compilation: numba

## Extensions 

There are a few ways to make python faster, which are called _extensions_. Typically, these boils down to either
- Write some core functions in compiled language like C or Fortran and interface it with python. This is also useful to recycle some legacy code written by the supervisor of your supervisor who doesn't know python.
- Use a library that converts python into compiled code (with some restrictions)

Some examples:

* **C**
  * [C-API](https://docs.python.org/3/c-api/index.html) : the
    standard python interpreter (cpython) is written in C, so it is
    natural that we can write C code to interact with our python code.
    This is the python C-API.  Since numpy is also written in C, we
    can work with numpy arrays in C code as well.
    This will give us the performance of C compiled code, but the
    downside is that we lose a lot of what makes python great.  We
    need to pass data into C as pointers and cast them into types that
    represent the arrays we use.  This means writing a lot of
    boilerplate code just to deal with some simple operations. These days, there are better methods for most applications.
  * [ctypes](https://docs.python.org/3/library/ctypes.html) : this
    is a module that allows you to call functions in shared libraries.
    This is part of standard python. With ctypes, you don't need to modify your C code -- you just need to
    define an interface to the C function in python.  However, the calling
    mechanism can be slow. There is support for numpy through numpy.ctypeslib.

* **Fortran**
  * [f2py](https://numpy.org/doc/stable/f2py/) : this is part of
    numpy.  It allows for easy calling of Fortran from python.
    You essentially just need to add some comments to your Fortran
    code to allow f2py to build an interface.  f2py understands the
    different orderings of indices between C and Fortran arrays.

* **python**
  * [Cython](https://cython.org/) : this is a superset of python that can convert python into
    compiled C code. The advantage here is that the code looks like python, with some
    declarations of the variable types with `cdef`.  Performance can be
    really great when you need to explicitly write out loops over
    numpy array indices. Advantage: you can identify the bottleneck of your code and cytonize only that part. 
  * [Numba](https://numba.pydata.org/) : this is a just-in-time
    compiler.  It just requires a simple decorator and then it will
    compile a python function the first time it is encountered.

* **GPU**: The modern (think deep learning) way of going faster is actually changing hardware and using GPUs instead of CPUs. That could probably fill an entire class, but [this](https://thedatafrog.com/en/articles/boost-python-gpu/) is a nice tutorial. There's even [some hardware](https://cloud.google.com/tpu/docs/intro-to-tpu) that is specifically built for AI (Google builds it to run TensorFlow).
    


Just-in-time compilation seems to be the cool thing to do these dats, so here we'll look at Numba. But before that... 

## Python decorators

With numba, the compilation of a python function is triggered by a decorator.

A python decorator is a function that takes another function as input, modifies it, and returns the modified function to the user. I realize that this sentence sounds tricky, but it's not. 

```
def decorator(function):
    [do something]
    output=function(input)
    [do something else]
    return [result]
```

Then if you have any function

```
def myprecious(ring):
    return gollum
```       
 
You can redefine it to e.g.

```
myprecious=decorator(myprecious)
```

or equivalently add

```
@decorator
def myprecious(ring):
    return gollum
```   

before the function call.
    


Remember that in python everything is an object. Functions are objects, and classes are objects too. For instance, let's take this simple function:

In [1]:
def hello():
  print('Hello world')

In [2]:
hello()

Hello world


`hello` is a function object, so we can pass it to another function like this one:

In [9]:
def make_sure(func):
  def wrapper():
    while 1:
      res = input('are you sure you want to greet the world? [y/n]')
      if res=='n':
        return
      elif res=='y':
        o = func()
        return o
  return wrapper

This is a decorator! `make_sure` takes an input function and returns a new function that wraps the input function.

Below, we decorate the function `hello`, and `whello` is the decorated function:

In [10]:
whello = make_sure(hello)
whello()


are you sure you want to greet the world? [y/n]
are you sure you want to greet the world? [y/n]
are you sure you want to greet the world? [y/n]
are you sure you want to greet the world? [y/n]n


In [11]:
@make_sure
def justtwo():
    return 2

In [12]:
justtwo()

are you sure you want to greet the world? [y/n]y


2

Of course, we can use the make_sure decorator on any function that has the same signature as `func` (can work without arguments, and no need to retrieve the output).

We know enough about decorators to use numba. Just one word about the syntax: we can also decorate a function in this way:

In [13]:
@make_sure
def hello():
  print('Hello world')

hello()

are you sure you want to greet the world? [y/n]n


There is really nothing mysterious about this, it's just a nice and easy syntax to decorate the function as soon as you write it.

## Just-in-time compilation with numba


Numba is able to compile python code into bytecode optimized for your machine, with the help the LLVM library. You don't really need to know what LLVM is to follow this tutorial, but here is a [nice introduction to LLVM](https://www.infoworld.com/article/3247799/what-is-llvm-the-power-behind-swift-rust-clang-and-more.html) in case you're interested.

Here is a function that can take a bit of time. This function takes a list of numbers, and returns the standard deviation of these numbers.

In [14]:
import math

def std(xs):
  # compute the mean
  mean = 0
  for x in xs:
    mean += x
  mean /= len(xs)
  # compute the variance
  ms = 0
  for x in xs:
     ms += (x-mean)**2
  variance = ms / len(xs)
  std = math.sqrt(variance)
  return std

As we can see in the code, we need to loop twice on the sample of numbers: first to compute the mean, and then to compute the variance, which is the square of the standard deviation.

Obviously, the more numbers in the sample, the more time the function will take to complete. Let's start with 10 million numbers, drawn from a Gaussian distribution of unit standard deviation:

In [17]:
import numpy as np
a = np.random.normal(0, 1, 10000000)

In [24]:
std(a)

0.9999433547917166

The function takes a second or so to compute the standard deviation of the sample.

Now, let's import the njit decorator from numba, and decorate our std function to create a new function:

In [26]:
from numba import njit
c_std = njit(std)

In [27]:
c_std(a)

0.9999433547917166

The performance improvement might not seem striking, but that's because the first time the function is called, numba will need to compile the function, which takes a bit of time.

But we can quantify the improvement using the timeit magic function, first for the interpreted version of the std function, and then for the compiled version:

In [28]:
%timeit std(a)

1.52 s ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit c_std(a)

17.6 ms ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The compiled function is 100 times faster!

But obviously we did not have to go into such trouble to compute the standard deviation of our array. For that, we can simply use numpy:

In [12]:
a.std()

0.9997743508708942

In [13]:
%timeit a.std()

13 ms ± 357 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In this particular case numba and numpy are comparable, but we'll see numba is much more flexible.

## Calculation of $\pi$

The number $\pi$ can be estimated with a very elegant Monte Carlo method.

Just consider a square of side L=2, centred on (0,0). In this square, we fit a circle of radius R=1, as shown in the figure below.

![](https://raw.githubusercontent.com/cbernet/maldives/master/numba/pi_mc.png)


The ratio of the circle area to the square area is

$$r = \frac{A_c}{A_s} = \frac{\pi R^2}{L^2} = \pi / 4$$

so

$$\pi = 4 r$$

So if we can estimate this ratio, we can estimate pi!

And to estimate this ratio, we will simply shoot a large number of points in the square, following a uniform probability distribution. The fraction of the points falling in the circle is an estimator of r.

Obviously, the more points, the more precise this estimator will be, and the more decimals of pi can be computed.

Let's implement this method, and use it with an increasing number of points to see how the precision improves.

(curious about how this works in [infinitely many dimensions](https://davidegerosa.com/nsphere/)?)

In [30]:
import random

def pi(npoints):
  n_in_circle = 0
  for i in range(npoints):
    x = random.random()
    y = random.random()
    if (x**2+y**2 < 1):
      n_in_circle += 1
  return 4*n_in_circle / npoints

In [31]:
npoints = [10, 100, 10000, int(10e6)]
for number in npoints:
  print(pi(number))


3.6
3.08
3.1232
3.142016


As you can see, even with N=10 million points, the precision is not great. More specifically, the relative uncertainty on pi can be calculated as

$$\delta = 1/ \sqrt{N}$$

(If you don't know why go back to your stats textbook)

Here is how the uncertainty evolves with the number of points:

In [16]:
import math
# defining the uncertainty function
# with a lambda construct
uncertainty = lambda x: 1/math.sqrt(x)
for number in npoints:
  print('npoints', number, 'delta:', uncertainty(number))

npoints 10 delta: 0.31622776601683794
npoints 100 delta: 0.1
npoints 10000 delta: 0.01
npoints 10000000 delta: 0.00031622776601683794


Clearly, we'll need a lot of points. How fast is our code?

In [32]:
%timeit pi(10000000)

1.24 s ± 2.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


A few seconds for 10 million points. This algorithm is O(N), so if we want to use 1 **billion** points, it will take us between 5 and 10 minutes . We don't have that much time, so let's use numba!

In [33]:
@njit
def fast_pi(npoints):
  n_in_circle = 0
  for i in range(npoints):
    x = random.random()
    y = random.random()
    if (x**2+y**2 < 1):
      n_in_circle += 1
  return 4*n_in_circle / npoints

In [35]:
fast_pi( int(1e9) )

3.14152802

This took <10 s, instead of 10 minutes!

## Finding the closest two points

Numpy features an efficient implementation for most array operations. 

My suggestion: **If numpy can do it, just go for it.** Once you've abandoned your C mindeset of using indexes for everything, numpy is just easier to write, easier to read, the world is shining, and everything is beautiful again. 

But sometimes, you'll come up with an expensive algorithm that cannot easily be implemented with numpy. For instance, let's consider the following function, which takes an array of 2D points, and looks for the closest two points.

In [36]:
import sys
def closest(points):
  '''Find the two closest points in an array of points in 2D.
  Returns the two points, and the distance between them'''

  # we will search for the two points with a minimal
  # square distance.
  # we use the square distance instead of the distance
  # to avoid a square root calculation for each pair of
  # points

  mindist2 = 999999.
  mdp1, mdp2 = None, None
  for i in range(len(points)):
    p1 = points[i]
    x1, y1 = p1
    for j in range(i+1, len(points)):
      p2 = points[j]
      x2, y2 = p2
      dist2 = (x1-x2)**2 + (y1-y2)**2
      if dist2 < mindist2:
        # squared distance is improved,
        # keep it, as well as the two
        # corresponding points
        mindist2 = dist2
        mdp1,mdp2 = p1,p2
  return mdp1, mdp2, math.sqrt(mindist2)

You might be thinking that this algorithm is quite naive, and it's true! It was written like this on purpose.

You can see that there is a double loop in this algorithm. So if we have N points, we would have to test NxN pairs of points, so the computational complexity here is $O(N^2)$.

To improve the situation a bit, please note that the distance between point i and point j is the same as the distance between point j and point i!
So there is no need to check this combination twice. Also, the distance between point i and itself is zero, and should not be tested...That's why we started the inner loop at i+1. So the combinations that are tested are:

* (0,1), (0,2), ... (0, N)
* (1,2), (1,3), ... (1, N)
* ...

Another thing to note is that we're doing all we can to limit the amount of computing power needed for each pair. That's why it's minimizing the square distance instead of the distance itself, which saves us a call to math.sqrt for every pair of points.

Still, the algorithm remains $O(N^2)$.

Let's first run this algorithm on a small sample of 10 points, just to check that it works correctly.

In [37]:
points = np.random.uniform((-1,-1), (1,1), (10,2))
print(points)
closest(points)

[[ 0.55545645  0.19532461]
 [ 0.93333211 -0.26740319]
 [ 0.21600159 -0.56871916]
 [ 0.79935586  0.26271225]
 [ 0.3757482  -0.93091315]
 [ 0.18387115  0.74700331]
 [-0.9426233  -0.13333073]
 [ 0.41215167  0.83672734]
 [ 0.56249722  0.31307642]
 [-0.3422276  -0.18140783]]


(array([0.55545645, 0.19532461]),
 array([0.56249722, 0.31307642]),
 0.11796211545817499)

Ok, this looks right, the two points indeed appear to be quite close. Let's see how fast is the calculation:

In [38]:
%timeit closest(points)

26.8 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Now, let's increase a bit the number of points in the sample. You will see that the calculation will be much slower.

In [39]:
points = np.random.uniform((-1,-1), (1,1), (2000,2))
closest(points)

(array([0.84324155, 0.87772559]),
 array([0.84338028, 0.87746215]),
 0.00029773565155952707)

In [40]:
%timeit closest(points)

1.01 s ± 8.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Since our algorithm is O(NxN), if we go from 10 to 2,000 points, the algorithm will be 200x200 = 40,000 times slower.

Now let's try and speed this up with numba's just in time compilation:

In [41]:
c_closest = njit(closest) # This is equivalent to decorating with @
c_closest(points)

(array([0.84324155, 0.87772559]),
 array([0.84338028, 0.87746215]),
 0.00029773565155952707)

In [34]:
c_closest(points)

(array([-0.99333223,  0.2580123 ]),
 array([-0.99360315,  0.25866478]),
 0.0007064888169048076)

In [42]:
%timeit closest(points)

1.02 s ± 6.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%timeit c_closest(points)

8.82 ms ± 55.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Again, the compiled code is 100 times faster! And this time we don't have a numpy alternative.

# Part 2. Parallel computing: multiprocessing

Why running code in parallel? To go faster.

Ultimately the number of operations one can do is set by the chip.


That said, things are flattening out ("Moore law is dead") and powerful chips are expensive.

Two modern solutions:

1. Break down your computation and use more chips. That's **parallel computing**.  Two ways, depending on your problem.
    - HPC (High Performance Computing). That's the real parallelization business, where you break down your computation into chunks, send the chunks to different processors, and (crucially) these processors need to talk to each other. For instance, solving a complicated PDE using subdomains, you need to exchange information on the boundary conditions. 
    - HTC (High Throughput Computing), also called "embarrassing" parallelization. Here the various jobs don't need to talk to each other. For instance a parameter-space study. You need to perform a costly operation many times, and all of these are independent of each other. 
    - .
    
2. Use **different hardware**. We're not going to go into this but you might have heard of things like:
    - GPU. That's the thing that renders your screen (the market is indeed dominated by gaming!), which it turns our it's very good at dealing with linear algebra. Deep learning applications are essentially a pile of matrices.
    - TPU. That's Google's hardware which is highly optimized to run Google's TensorFlow pipelines.
    - FPGA. Programmable hardware. I don't know anything about this.


Let's look at **embarrassingly parallel programming** with python, hope this is useful for some of your research needs.

For true HPC in python, look into things like [mpi4py](https://mpi4py.readthedocs.io/en/stable/)

One can parallelize things using multiple cores of a single machine, or using multiple machines.
 - Single-node parallelization means using all the cores in a chip (can be useful even on your laptop!)
 - Multi-node parallelization, well, requires multiple computers that are appropriately linked together. Say in a big cluster (is any of you using Galileo or Leonardo at CINECA?)


In python, embarassingle parallelization on a single node is done with the **multiprocessing** package. Here we use a wrapper provided in the 3rd-party **pathos** package. It's the same, just a bit more user fiendly IMO.

In [44]:
#You might need to pip install pathos and tqdm

import numpy as np
import multiprocessing, pathos.multiprocessing
from tqdm.notebook import tqdm
import os
import time

I want to execute a function...

In [46]:
def fun(a,b):
    #print(os.getpid())
    time.sleep(1) # So that it takes a bit longer...
    return a+b

... many times.

In [51]:
N=10
a=np.linspace(0,1,N)
b=np.linspace(0,1,N)

In [52]:
a,b

(array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
 array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]))

I can do it with a for loop...

In [55]:
%%time

aplusb=[]
for ax,bx in zip(a,b):
    aplusb.append(fun(ax,bx))


CPU times: user 2.01 ms, sys: 1.31 ms, total: 3.32 ms
Wall time: 10 s


In [56]:
aplusb

[0.0,
 0.2222222222222222,
 0.4444444444444444,
 0.6666666666666666,
 0.8888888888888888,
 1.1111111111111112,
 1.3333333333333333,
 1.5555555555555554,
 1.7777777777777777,
 2.0]

(of course this is a stupid example and you should do it with numpy! But let's say `fun` is a complicate thing of thousands of lines)

We can implement the same thing using python's map function. This is not going to be any faster, it's just a compact way of writing a for loop.

In [57]:
%%time 

aplusb = list(  map(fun, a,b)  )


CPU times: user 1.89 ms, sys: 1.9 ms, total: 3.79 ms
Wall time: 10 s


... do it again, just adding a nice progress bar

In [58]:
%%time 

aplusb = list(tqdm(map(fun, a,b),total=N))
print(aplusb)

  0%|          | 0/10 [00:00<?, ?it/s]

[0.0, 0.2222222222222222, 0.4444444444444444, 0.6666666666666666, 0.8888888888888888, 1.1111111111111112, 1.3333333333333333, 1.5555555555555554, 1.7777777777777777, 2.0]
CPU times: user 32 ms, sys: 12.9 ms, total: 45 ms
Wall time: 10.1 s


Now let's try to make full use of my computer. How many cores do I have on my chip?

In [59]:
multiprocessing.cpu_count()

8

python's map syntax can be easily generalized to use multiple cores. Let's use a few of them, all at once...

In [64]:
%%time 

CPUS = 4

# Now, this is much faster!
parmap = pathos.multiprocessing.ProcessingPool(CPUS).imap


aplusb = list(tqdm(parmap(fun, a,b),total=N))

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 26.6 ms, sys: 8.26 ms, total: 34.9 ms
Wall time: 3.03 s


That's faster! And with minimal editing to the code (just define parmap and change map --> parmap)

In [45]:
# aplusb = list( tqdm( map(    fun, a,b), total=N))
# aplusb = list( tqdm( parmap( fun, a,b), total=N))

Note the scaling is not perfectly linear, because there's an overhead when spawning new processes

# Exercises

For the exam prepare two of these exercises.

## Q1: The stock market

(This is about numba)

A Markov Chain is defined as a sequence of random variables where a parameter depends *only* on the preceding value. This is a crucial tool in statistics, widely used in science and beyond (economics for instance).

For instance, the stock market has phases of growing prices (bull), dreasing prices (bear) and recession. This would be a Markov Chain model:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Finance_Markov_chain_example_state_space.svg/400px-Finance_Markov_chain_example_state_space.svg.png)

where the numbers on the arrows indicate the probability that the next day will be in a given state.

Your task is to simulate the stock market according to this rule. Start from a random state and simulate many many  iterations. If your code is right, the fraction of days in each state should converge. 

Implement a pure-python version and a numba version, and compare speeds. 


## Q2: Consistent plotting

(This is about python's dectorators)

Write a decorator for the plots of all your papers. 

- Remember a decorator takes a function. 
- This function in turn should return a matplotlib figure object.
- Before the function is called, the decorator should initialize a matplotlib figure with the options that you like the most (fontsize, ticks, etc etc)
- After the figure it's done, the decorator should save it to pdf.

This is a great hack for your papers! You do this once and for all, and all plots in your paper will be beautifull, all with the same style/fontsize/etc. All you'll need to do is adding `@myplot` to the relevant plotting functions. 
    
The decorator that I use for my papers is available in my python module [skywalker](https://github.com/dgerosa/skywalker).  

## Q3: Scaling

(This is about multiprocessing)

The ["scaling"](https://hpc-wiki.info/hpc/Scaling) of a code refers to its performance of as a function of the number of cores adopted. 

- Define a computationally intensive task (something like an operation on two giant arrays with >1e7 numbers or, even better!, pick somethinbg from your research). 
- Make sure it's embarrassingly parallel. 
- Implement a parallelization strategy using multiprocessing. 
- Plot the time the code takes as a function of the number of cores.
- Figure out the number of cores in your CPU and make sure the plot extends both below and above this number.
- Interpret the resulting features. 
- A perfect scaling results in straight line (linear dependency). How perfect is your scaling?

### Important
Numpy has some inner, semi-automatic parallelization functionalities. Some, but not all, numpy functions detect the number of CPUs in your machine and make good use of them. That's great for most applications, but when performing a scaling study you want to control the parallelization yourself and disable what's done by numpy's. The following forces numpy to use a single core.  

In [47]:
import numpy as np
np.__config__.show()
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

openblas64__info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None), ('BLAS_SYMBOL_SUFFIX', '64_'), ('HAVE_BLAS_ILP64', None)]
    runtime_library_dirs = ['/usr/local/lib']
blas_ilp64_opt_info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None), ('BLAS_SYMBOL_SUFFIX', '64_'), ('HAVE_BLAS_ILP64', None)]
    runtime_library_dirs = ['/usr/local/lib']
openblas64__lapack_info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None), ('BLAS_SYMBOL_SUFFIX', '64_'), ('HAVE_BLAS_ILP64', None), ('HAVE_LAPACKE', None)]
    runtime_library_dirs = ['/usr/local/lib']
lapack_ilp64_opt_info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None